<a href="https://colab.research.google.com/github/mm230/Text-Classification-using-Fully-connected-FC-NN/blob/main/text_classification_project_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from collections import Counter
from pyparsing import WordStart
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import requests
from io import StringIO
import math

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [77]:
!unzip "/content/drive/MyDrive/Text_classification_FCN/archive.zip" -d "/content/drive/MyDrive/Text_classification_FCN"

Archive:  /content/drive/MyDrive/Text_classification_FCN/archive.zip
replace /content/drive/MyDrive/Text_classification_FCN/reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/MyDrive/Text_classification_FCN/reviews.csv  


In [78]:
data = pd.read_csv("/content/drive/MyDrive/Text_classification_FCN/reviews.csv")

In [79]:
data.shape

(12495, 12)

In [80]:


data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOEhZuqSqqWnaKRgv-9ABYdajFUB0WugPGh-SG-...,Eric Tie,https://play-lh.googleusercontent.com/a-/AOh14...,I cannot open the app anymore,1,0,5.4.0.6,2020-10-27 21:24:41,NaN,NaN,newest,com.anydo
1,gp:AOqpTOH0WP4IQKBZ2LrdNmFy_YmpPCVrV3diEU9KGm3...,john alpha,https://play-lh.googleusercontent.com/a-/AOh14...,I have been begging for a refund from this app...,1,0,NaN,2020-10-27 14:03:28,"Please note that from checking our records, yo...",2020-10-27 15:05:52,newest,com.anydo
2,gp:AOqpTOEMCkJB8Iq1p-r9dPwnSYadA5BkPWTf32Z1azu...,Sudhakar .S,https://play-lh.googleusercontent.com/a-/AOh14...,Very costly for the premium version (approx In...,1,0,NaN,2020-10-27 08:18:40,NaN,NaN,newest,com.anydo
3,gp:AOqpTOGFrUWuKGycpje8kszj3uwHN6tU_fd4gLVFy9z...,SKGflorida@bellsouth.net DAVID S,https://play-lh.googleusercontent.com/-75aK0WF...,"Used to keep me organized, but all the 2020 UP...",1,0,NaN,2020-10-26 13:28:07,What do you find troublesome about the update?...,2020-10-26 14:58:29,newest,com.anydo
4,gp:AOqpTOHls7DW8wmDFzTkHwxuqFkdNQtKHmO6Pt9jhZE...,Louann Stoker,https://play-lh.googleusercontent.com/-pBcY_Z-...,Dan Birthday Oct 28,1,0,5.6.0.7,2020-10-26 06:10:50,NaN,NaN,newest,com.anydo


In [81]:
data = data[['content', 'score']]

In [82]:
class PreprocessData:
    def __init__(self,path,lang='english'): 
        self.dataset=pd.read_csv(path, sep=",")
        self.dataset=self.dataset[["content","score"]]
        self.stopwords=stopwords.words(lang)
        self.preprocess()
    
    @staticmethod
    def remove_punctuation(text):
        '''a function for removing punctuation'''
        # replacing the punctuations with no space, 
        # which in effect deletes the punctuation marks 
        translator = str.maketrans('', '', string.punctuation)
        # return the text stripped of punctuation marks
        return text.translate(translator)

    #A function to remove the stopwords
    def remove_stopwords(self,text):
        text = [word.lower() for word in text.split() if word.lower() not in self.stopwords]
        # joining the list of words with space separator
        return " ".join(text)

    def preprocess(self):
        self.dataset.iloc[:,0] = self.dataset.iloc[:,0].apply(self.remove_punctuation)
        self.dataset.iloc[:,0] = self.dataset.iloc[:,0].apply(self.remove_stopwords)

In [83]:
path = "/content/drive/MyDrive/Text_classification_FCN/reviews.csv"

In [84]:
dataset=PreprocessData(path)

In [85]:
dataset.dataset.head()

,content,score
0,cannot open app anymore,1
1,begging refund app month nobody replying,1
2,costly premium version approx indian rupees 91...,1
3,used keep organized 2020 updates made mess thi...,1
4,dan birthday oct 28,1


## Tokenization

In [101]:
def tokenize(data): 
  word = []
  for i in range(len(data)):
    new_doc = data[i].split()
    word.append(new_doc)
  return word

In [102]:
token = tokenize(dataset.dataset['content'])

In [122]:
token

[['cannot', 'open', 'app', 'anymore'],
 ['begging', 'refund', 'app', 'month', 'nobody', 'replying'],
 ['costly',
  'premium',
  'version',
  'approx',
  'indian',
  'rupees',
  '910',
  'per',
  'year',
  'better',
  'download',
  'premium',
  'version',
  'app',
  'apkmos',
  'website',
  'use',
  'microsoft',
  'list',
  'app',
  'far',
  'better'],
 ['used',
  'keep',
  'organized',
  '2020',
  'updates',
  'made',
  'mess',
  'things',
  'cudnt',
  'u',
  'leave',
  'well',
  'enuf',
  'alone',
  'guess',
  'ur',
  'techies',
  'feel',
  'need',
  'keep',
  'making',
  'changes',
  'justify',
  'continuing',
  'collect',
  'salary',
  '🤤🤤🤤'],
 ['dan', 'birthday', 'oct', '28'],
 ['changed',
  'viewed',
  'different',
  'lists',
  'jumbled',
  'together',
  'cant',
  'find',
  'need'],
 ['im',
  'looking',
  'grocery',
  'list',
  'app',
  'every',
  'time',
  'tap',
  'away',
  'app',
  'tap',
  'opening',
  'see',
  'list',
  'cant',
  'find',
  'way',
  'keep',
  'certain',
  'lis

In [133]:
import gensim.models 
model = gensim.models.Word2Vec(sentences=token, workers = 1, size = 50, min_count = 1, window = 3)

## Word2vec

In [59]:
import gensim

In [144]:
model.wv.vectors.shape

(13948, 50)

In [121]:
ims = model.wv.most_similar('app', topn=10)
ims

[('far', 0.9849435687065125),
 ('idea', 0.9747955799102783),
 ('really', 0.974092423915863),
 ('pretty', 0.9717373847961426),
 ('concept', 0.9707658886909485),
 ('looks', 0.9688400030136108),
 ('overall', 0.9651016592979431),
 ('interface', 0.9608306884765625),
 ('job', 0.9595315456390381),
 ('looking', 0.9589308500289917)]

In [139]:
model.wv

In [151]:
class WordRepresentation:
    def __init__(self,dataset,wv):
        self.dataset=dataset
        self.glove={}
        self.word2vec={}
        # self.compute_glove()
        self.compute_word2vec(wv)
    
    #One hot
    def compute_word2vec(self, wv):
        if self.glove:
            #The case if we have the ponderations or the frequencies , we don't need to search the words again
            #This condition will reduce the time complexity a lot   
            for word in self.glove:
              if word in wv.vocab:
                self.word2vec[word]=wv.word_vec(word)
        else:
          for i in range(len(self.dataset)):
            document=self.dataset.iloc[i,0]
            word_list=document.split(" ")
            for word in word_list:
              if word in wv.vocab:
                self.word2vec[word]=wv.word_vec(word)

In [153]:
test = WordRepresentation(dataset.dataset.iloc[:20], model.wv)

In [154]:
test.word2vec

{'101': array([ 0.02477635,  0.01385824,  0.00753359, -0.00724155, -0.01996606,
        -0.02205929, -0.00030513,  0.00592715,  0.01949078, -0.01370617,
         0.00788546,  0.01698669,  0.03600529, -0.01207288,  0.01685584,
         0.03764801, -0.00934698,  0.01189901, -0.00565042,  0.02020028,
        -0.03679663,  0.01198522,  0.00947744, -0.02021796,  0.00449395,
         0.03635353, -0.03828001,  0.0009032 , -0.01883725,  0.00619804,
        -0.00296837,  0.0364638 ,  0.0001545 ,  0.01308178, -0.05623578,
         0.02681586,  0.0139126 ,  0.00866753, -0.00253981, -0.01234971,
        -0.01777149,  0.01167504,  0.0017391 , -0.07852215,  0.04382066,
         0.0124948 ,  0.00362813, -0.00341531,  0.01892235, -0.02332998],
       dtype=float32),
 '2020': array([ 0.23130994,  0.19192305,  0.0987478 , -0.02165319, -0.13622162,
        -0.28148195,  0.02418505,  0.01905307,  0.16706665, -0.1733494 ,
         0.02089093,  0.28065372,  0.32943562, -0.15797085,  0.13217753,
         0.3